# Setep 5: Machine Learning Model

<br>
Clean up any values left from any previous steps

In [6]:
#Quick clean up
for name in dir():
    if not name.startswith('_'): # and name not in ['mbti_FE','mbti_Dataset', 'full_Lem_CV', 'full_Lem_Ngram', 'full_Lem_tfidf']:
        del globals()[name]

Load Dataset and results from previous steps.

In [1]:
#Load information from prevous steps
import pandas as pd
import numpy as np
from scipy import sparse

mbti_Dataset = pd.read_csv('mbti_Dataset.csv')
#mbti_FE = pd.read_csv('mbti_FE.csv')

#full_Lem_CV = sparse.load_npz('full_Lem_CV.npz')
#full_Lem_Ngram = sparse.load_npz('full_Lem_Ngram.npz')
#full_Lem_tfidf = sparse.load_npz('full_Lem_tfidf.npz')
features_Dic = {'Count Vectorizer': sparse.load_npz('full_Lem_CV.npz'), 
                'TFIDF Vectorizer': sparse.load_npz('full_Lem_tfidf.npz')}

#np.save('full_Lem_CV', full_Lem_CV.toarray())
#np.save('f:/full_Lem_Ngram', full_Lem_Ngram.toarray())
#np.save('full_Lem_tfidf', full_Lem_tfidf.toarray())

## 5-1: Random Forest Model
### 5-1-1: Explorering Random Forest with Holdout test set + grid-search

In [69]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_rows', None, 'display.max_colwidth', 500)

def rfClassifier_GridSearch(X_Features, dataset_PD, test_Szie, Lable, feature_type, n_est, depth, is_print = True,):
    if Lable == 'IE':
        Lable = 'I'
        predict_Lable = 'Introverts'
    elif Lable == 'NS':
        Lable = 'N'
        predict_Lable = 'Intuitives'
    elif Lable == 'FT':
        Lable = 'F'
        predict_Lable = 'Feelers'
    elif Lable == 'PJ':
        Lable = 'P'
        predict_Lable = 'Perceivers'

    X_train, X_test, Y_train, Y_test = train_test_split(X_Features, dataset_PD, test_size=test_Szie)#20% of dataset is test set
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth ,n_jobs=-1)#Max depth of tree is 20
    rf_model = rf.fit(X_train, Y_train)
    Y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(Y_test, Y_pred, pos_label=Lable, average='binary')
    if is_print:
        print('Being ' + predict_Lable + ' using ' + feature_type + ':'  ,
        'Estimators: {} / Max_Depth: {} --> Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        n_est,
                                                        depth,
                                                        round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test).sum() / len(Y_pred),3)))
    return([predict_Lable, feature_type, n_est, depth, precision, recall, round((Y_pred==Y_test).sum() / len(Y_pred),3)])



classes = ['IE' , 'NS', 'FT', 'PJ']
rf_result = []

for item in classes:
    for key, X_Features in features_Dic.items():
        for n_est in [10, 50, 100]:
            for depth in [10, 20, 30, None]:
                rf_result.append(rfClassifier_GridSearch(X_Features, mbti_Dataset[item], 0.2, item, key, n_est, depth, is_print=False))

pd.DataFrame(rf_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Precision', 'Recall', 'Accuracy']).to_csv('RF_Holdout_Result.csv')
pd.DataFrame(rf_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Precision', 'Recall', 'Accuracy'])

,Type,Method,Estimators,Max_Depth,Precision,Recall,Accuracy
0,Introverts,Count Vectorizer,10,10.0,0.761383,1.000000,0.761
1,Introverts,Count Vectorizer,10,20.0,0.768786,0.999249,0.769
2,Introverts,Count Vectorizer,10,30.0,0.758242,0.996958,0.757
3,Introverts,Count Vectorizer,10,NaN,0.778708,0.978212,0.770
4,Introverts,Count Vectorizer,50,10.0,0.764841,1.000000,0.765
5,Introverts,Count Vectorizer,50,20.0,0.769452,1.000000,0.769
6,Introverts,Count Vectorizer,50,30.0,0.774063,1.000000,0.774
7,Introverts,Count Vectorizer,50,NaN,0.774957,1.000000,0.775
8,Introverts,Count Vectorizer,100,10.0,0.776945,1.000000,0.777
9,Introverts,Count Vectorizer,100,20.0,0.763689,1.000000,0.764


### 5-1-2: Evaluation Random Forest Model

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

pd.set_option('display.max_colwidth', 500)

def RF_Evaluation(param, X_Features, dataset_PD, lable, is_print = True):
    rf = RandomForestClassifier()
    gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)#cv=5 means 5 folde validation
    gs_fit = gs.fit(X_Features, dataset_PD)
    if is_print:
        print(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
        'param_n_estimators', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])

    return(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
    'param_n_estimators', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])


param = {'n_estimators' : [10, 150, 300],
        'max_depth' : [30, 60, 90, None]}

classes = ['IE' , 'NS', 'FT', 'PJ']
for item in classes:
    for key, X_Features in features_Dic.items():
        RF_Evaluation(param, X_Features, mbti_Dataset[item], item)
        print(key)


   param_max_depth param_n_estimators  std_test_score  mean_test_score  \
6               90                 10        0.004380         0.771873   
0               30                 10        0.001094         0.770605   
10            None                150        0.000565         0.770490   
11            None                300        0.000461         0.770375   
7               90                150        0.000461         0.770259   

    rank_test_score  
6                 1  
0                 2  
10                3  
11                4  
7                 5  
Count Vectorizer
   param_max_depth param_n_estimators  std_test_score  mean_test_score  \
10            None                150        0.000936         0.770259   
8               90                300        0.000431         0.770144   
11            None                300        0.000672         0.770144   
4               60                150        0.000365         0.770029   
5               60                30

## 5-2: Gradient Boosting Model
### 5-2-1: Gradient Boosting with Holdout test set

In [2]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
from datetime import datetime


pd.set_option('display.max_rows', None, 'display.max_colwidth', 500)


def GBoosting_GridSearch(X_Features, dataset_PD, test_Szie, Lable, feature_type , n_est, depth, lr ,is_print = True):
    if Lable == 'IE':
        Lable = 'I'
        predict_Lable = 'Introverts'
    elif Lable == 'NS':
        Lable = 'N'
        predict_Lable = 'Intuitives'
    elif Lable == 'FT':
        Lable = 'F'
        predict_Lable = 'Feelers'
    elif Lable == 'PJ':
        Lable = 'P'
        predict_Lable = 'Perceivers'
    X_train, X_test, Y_train, Y_test = train_test_split(X_Features, dataset_PD, test_size=test_Szie) # 20% of our dataset is test set
    gb = GradientBoostingClassifier(n_estimators=n_est, max_depth=depth , learning_rate=lr)#Max depth of tree is 20
    gb_model = gb.fit(X_train, Y_train)
    Y_pred = gb_model.predict(X_test)
    precision, recall, fscore, support = score(Y_test, Y_pred, pos_label=Lable, average='binary')
    if is_print:
        print('Being ' + predict_Lable + ' using ' + feature_type + ': '  ,'Estimators: {} / Max_Depth: {} / Learning_Rate: {} --> Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        n_est,
                                                        depth,
                                                        lr,
                                                        round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test).sum() / len(Y_pred),3)))
    return([predict_Lable, feature_type, n_est, depth, lr, precision, recall, round((Y_pred==Y_test).sum() / len(Y_pred),3)])



classes = ['IE' , 'NS', 'FT', 'PJ']
gb_result = []

for item in classes:
    for key, X_Features in features_Dic.items():
        for n_est in [50, 100, 150]:
            for depth in [3, 7, 11, 15]:
                for lr in [0.01, 0.10, 1.00]:
                    gb_result.append(GBoosting_GridSearch(X_Features, mbti_Dataset[item], 0.2, item, key, n_est, depth, lr, is_print=False))
                    print('Moving to {} as Learning_Rate at: '.format(lr), datetime.now())
                print('Moving to {} as Max_Depth at: '.format(depth), datetime.now())
            print('Moving to {} estimators at: '.format(n_est), datetime.now())
        print('Moving to {} feature at: '.format(key), datetime.now())
    print('Moving to {} class at: '.format(item), datetime.now())

pd.DataFrame(gb_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Learning_Rate','Precision', 'Recall', 'Accuracy']).to_csv('GB_Holdout_Result.csv')
pd.DataFrame(gb_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Learning_Rate','Precision', 'Recall', 'Accuracy']).head()


ving to 0.1 as Learning_Rate at:  2021-06-26 02:08:21.947917
Moving to 1.0 as Learning_Rate at:  2021-06-26 02:20:05.114699
Moving to 11 as Max_Depth at:  2021-06-26 02:20:05.114699
Moving to 0.01 as Learning_Rate at:  2021-06-26 02:48:08.762527
Moving to 0.1 as Learning_Rate at:  2021-06-26 03:05:06.819204
Moving to 1.0 as Learning_Rate at:  2021-06-26 03:20:12.301838
Moving to 15 as Max_Depth at:  2021-06-26 03:20:12.301838
Moving to 150 estimators at:  2021-06-26 03:20:12.301838
Moving to TFIDF Vectorizer feature at:  2021-06-26 03:20:12.301838
Moving to IE class at:  2021-06-26 03:20:12.301838
Moving to 0.01 as Learning_Rate at:  2021-06-26 03:21:31.333904
Moving to 0.1 as Learning_Rate at:  2021-06-26 03:22:50.244996
Moving to 1.0 as Learning_Rate at:  2021-06-26 03:24:07.262245
Moving to 3 as Max_Depth at:  2021-06-26 03:24:07.262245
Moving to 0.01 as Learning_Rate at:  2021-06-26 03:26:22.749736
Moving to 0.1 as Learning_Rate at:  2021-06-26 03:28:23.732490
Moving to 1.0 as Lear

,Type,Method,Estimators,Max_Depth,Learning_Rate,Precision,Recall,Accuracy
0,Introverts,Count Vectorizer,50,3,0.01,0.774640,1.000000,0.775
1,Introverts,Count Vectorizer,50,3,0.10,0.828499,0.973333,0.822
2,Introverts,Count Vectorizer,50,3,1.00,0.842327,0.891030,0.786
3,Introverts,Count Vectorizer,50,7,0.01,0.758083,0.999239,0.758
4,Introverts,Count Vectorizer,50,7,0.10,0.821705,0.960725,0.811


### 5-2-2: Evaluation Gradient Boosting Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

pd.set_option('display.max_colwidth', 500)


def GB_Evaluation(param, X_Features, dataset_PD, lable):
    gb = GradientBoostingClassifier()
    gs = GridSearchCV(gb, param, cv=5)#cv=5 meand 5 folde validation
    gs_fit = gs.fit(X_Features, dataset_PD)
    print(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
     'param_n_estimators', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])

    return(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
     'param_n_estimators', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])
  
param = {'n_estimators' : [50, 100, 150],
                'max_depth' : [7, 11, 15],
                'Learning_rate' : [0.1]}

classes = ['IE' , 'NS', 'FT', 'PJ']


for item in classes:
    for key, X_Features in features_Dic.items():
        GB_Evaluation(param, X_Features, mbti_Dataset[item], item)
        print(key)

In [ ]:
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)